# Analyze poor municipal bridges not in TYP

In [1]:
import pandas as pd

### Import latest PennDOT bridge data

In [54]:
bridges = pd.read_csv('input/onemap.csv', usecols=['BRKEY', 'REPORTGROUP', 'OWNER', 'CONDITION', 'PLACECODE', 'ADTTOTAL'])
bridges = bridges[(bridges.REPORTGROUP == 'L1') & (bridges.CONDITION == 'P')]

bridges['OWNER'] = bridges.OWNER.replace({
    2: 'COUNTY HIGHWAY AGENCY',
    3: 'BOROUGH-CITY-TOWNSHIP',
    4: 'BOROUGH-CITY-TOWNSHIP',
    25: 'OTHER LOCAL AGENCIES',
    27: 'RAILROAD'
})

In [55]:
bridges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348 entries, 41 to 12659
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ADTTOTAL     348 non-null    float64
 1   OWNER        348 non-null    object 
 2   PLACECODE    348 non-null    int64  
 3   REPORTGROUP  348 non-null    object 
 4   CONDITION    348 non-null    object 
 5   BRKEY        348 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 19.0+ KB


### Import construction projects

In [37]:
projects = pd.read_csv('input/projects.csv', usecols=['BRIDGE KEY'])
projects = pd.DataFrame(projects['BRIDGE KEY'].unique(), columns=['BRKEY'])

### Filter for bridges which have been "poor" since at least 2020

In [56]:
nbi2021 = pd.read_csv('input/nbi2021.csv', usecols=['STRUCTURE_NUMBER_008', 'BRIDGE_CONDITION'])

nbi2021 = nbi2021[
    (nbi2021.STRUCTURE_NUMBER_008.notna()) &
    (nbi2021.STRUCTURE_NUMBER_008.str.isdigit()) &
    (nbi2021.BRIDGE_CONDITION == 'P')
]

nbi2021['STRUCTURE_NUMBER_008'] = nbi2021['STRUCTURE_NUMBER_008'].astype(int)

/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_80080/63071385.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  nbi2021 = pd.read_csv('input/nbi2021.csv', usecols=['STRUCTURE_NUMBER_008', 'BRIDGE_CONDITION'])


In [57]:
lala = pd.merge(bridges, nbi2021, left_on='BRKEY', right_on='STRUCTURE_NUMBER_008', how='left', indicator='nbimerge')

In [58]:
lala.nbimerge.value_counts()

both          195
left_only     153
right_only      0
Name: nbimerge, dtype: int64

In [59]:
bridges = lala

### Filter for bridges with no scheduled construction

In [60]:
bridges = pd.merge(
    bridges, projects,
    on='BRKEY',
    how='left',
    indicator=True
)

In [61]:
bridgesummary = bridges._merge.value_counts().to_frame()
bridgesummary['pct'] = bridgesummary._merge / bridgesummary._merge.sum()
bridgesummary

,_merge,pct
left_only,285,0.818966
both,63,0.181034
right_only,0,0.000000


### Group by municipality

In [62]:
placecodes = pd.read_csv('input/placecode.csv')

In [72]:
bridgesbymuni = bridges[
    (bridges.OWNER == 'BOROUGH-CITY-TOWNSHIP') &
    (bridges.nbimerge == 'both')
].groupby('PLACECODE').apply(lambda x: pd.Series(dict(
    projCount=len(x[x._merge == 'both']),
    poorCount=len(x.BRKEY),
    projADT=x[x._merge == 'both'].ADTTOTAL.sum(),
    poorADT=x.ADTTOTAL.sum(),
)))

bridgesbymuni = pd.merge(bridgesbymuni, placecodes, on='PLACECODE')

bridgesbymuni['projPct'] = bridgesbymuni.projCount / bridgesbymuni.poorCount
bridgesbymuni['ADTPct'] = bridgesbymuni.projADT / bridgesbymuni.poorADT

bridgesbymuni[bridgesbymuni.poorCount > 1].sort_values(by=['poorADT', 'ADTPct'], ascending=False)

,PLACECODE,projCount,poorCount,projADT,poorADT,MUNICIPALITY,projPct,ADTPct
43,61000,12.0,19.0,76886.0,107223.0,02/301 - PITTSBURGH,0.631579,0.717066
56,78528,2.0,3.0,16580.0,16780.0,26/302 - UNIONTOWN,0.666667,0.988081
29,49920,3.0,5.0,5050.0,14050.0,02/445 - MILLVALE,0.600000,0.359431
42,60712,1.0,2.0,100.0,5100.0,02/453 - PITCAIRN,0.500000,0.019608
27,46256,1.0,2.0,0.0,3300.0,02/304 - MCKEESPORT,0.500000,0.000000
3,9432,0.0,2.0,0.0,1500.0,26/402 - BROWNSVILLE,0.000000,0.000000
14,15776,0.0,3.0,0.0,1476.0,26/301 - CONNELLSVILLE,0.000000,0.000000
38,58880,0.0,2.0,0.0,1050.0,64/102 - PENN,0.000000,0.000000
16,18968,0.0,2.0,0.0,850.0,64/204 - DERRY,0.000000,0.000000
8,11176,0.0,3.0,0.0,700.0,62/204 - CANTON,0.000000,0.000000


In [64]:
bridges[bridges.PLACECODE == 52432].BRKEY

85     37018
89     37016
104    37024
111    37013
165    37017
250    37023
Name: BRKEY, dtype: int64

In [71]:
nbi2021[nbi2021.STRUCTURE_NUMBER_008 == 37023]

,STRUCTURE_NUMBER_008,BRIDGE_CONDITION


In [73]:
br = pd.read_csv('input/onemap.csv')

/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_80080/298172630.py:1: DtypeWarning: Columns (9,46,57,83,85,87,102,104) have mixed types. Specify dtype option on import or set low_memory=False.
  br = pd.read_csv('input/onemap.csv')


In [87]:
# array(['P', 'A', 'C', 'B', 'D', nan, 'R'], dtype=object)
# br[br.TEMPSTRUC == 'T'].sort_values(by='ADTTOTAL', ascending=False)
br[br.LOCATION == '110 SWINBURNE BRIDGE'].to_csv('swin.csv', index=False)